---

# **Generating Synthetic Data using CTGAN**


https://github.com/sdv-dev/CTGAN

----

# **Showcases of synthetic data:**
## 1. **Input**: original .csv file, **output**: synthetic .csv file
## 2. **Input**: original database table, **output**: synthetic database table

---

## **1. Input: original .csv file, output: synthetic .csv file**

## 1.1 Script

In [ ]:
from sdv.demo import load_tabular_demo
from sdv.tabular import CTGAN
import pandas as pd
import ocifs
import os
from ocifs import OCIFileSystem

def script_1(input_data):
    
    #get the primary key (like Employee ID or customer ID), the csv name, and determine number of new synthetic rows.
    prim_key = data["prim_key"]
    csv_name = data["csv_name"]
    number_new_rows = data["number_new_rows"]
    
    #get the csv file from a bucket
    input_location = "oci://West_BP@frqap2zhtzbe/synthetic_data/"
    input_csv = pd.read_csv(input_location + csv_name)
    
    #show snapshot of original data
    print("--------------------------------")
    print("Snapshot of the original data is: ")
    print("--------------------------------")
    print(input_csv.head(10).to_string())

    #get max 200 rows to train on (for demo purposes)
    data_short = input_csv.head(200)
    
    #load ctgan model
    model = CTGAN(primary_key=prim_key)
    
    #fit model on short data
    model.fit(data_short)
    
    #output file name
    output_file_name = "/home/datascience/synthetic_" + csv_name
    
    #delete file if exists already
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
    else:
        print("The file does not exist yet, but that's fine")
    
    #create new synthetic rows and store as .csv file
    new_data = model.sample(num_rows = number_new_rows, output_file_path = output_file_name) 
    
    #copy new .csv file back to bucket
    fs = OCIFileSystem()
    fs.invalidate_cache()
    
    new_csv_local_path = os.path.join("/home/datascience/", output_file_name)
       
    with open(new_csv_local_path, 'rb') as f:
        with fs.open(input_location + os.path.basename(new_csv_local_path), 'wb') as file_out:
            file_out.write(f.read())
    
    #show snapshot of synthetic data
    print("--------------------------------")
    print("--------------------------------")
    print("--------------------------------")
    print("Snapshot of the synthetic data is: ")
    print("--------------------------------")
    loc_synthetic_data = "oci://West_BP@frqap2zhtzbe/synthetic_data/" + "synthetic_" + csv_name
    snapshot_synthetic = pd.read_csv(loc_synthetic_data)
    print(snapshot_synthetic.head(10).to_string())
    
    print("--------------------------------")
    print("--------------------------------")
    return print("Synthetic data has been generated")

## 1.2 Run script and see output

In [ ]:
# determine values to pass 
primary_key = "PassengerId"
csv_name = "titanic_original.csv"
number_new_rows = 200

#call script_1
data = {"prim_key":primary_key, "csv_name":csv_name, "number_new_rows":number_new_rows}
script_1(data)

## **2. Input: original database table, output: synthetic database table**

## 2.1 Script

In [ ]:
password = "xx"

In [ ]:
from sdv.demo import load_tabular_demo
from sdv.tabular import CTGAN
import pandas as pd
import ocifs
import os
from ocifs import OCIFileSystem
import ads
    
def script_2(input_data):

    #get the primary key (like Employee ID or customer ID), the csv name, and determine number of new synthetic rows.
    prim_key = data["prim_key"]
    table_name = data["table_name"]
    number_new_rows = data["number_new_rows"]

    #connect to the autonomous database
    connection_parameters = {
        "user_name": "BOB",
        "password": password,
        "service_name": "adwwest_high",
        "wallet_location": "/home/datascience/synthetic_data/wallet/Wallet_ADWWEST.zip"}
    
    # Read and show titanic data from database
    sql_statement = "SELECT * FROM " + table_name
    df_original = pd.DataFrame.ads.read_sql(sql_statement, connection_parameters=connection_parameters)
    
    #show snapshot of original data
    print("--------------------------------")
    print("--------------------------------")
    print("--------------------------------")
    print("Snapshot of the original data is: ")
    print("--------------------------------")
    print(df_original.head(10).to_string())
    
    #get max 200 rows to train on (for demo purposes)
    data_short = df_original.head(200)
    
    #load ctgan model
    model = CTGAN(primary_key=prim_key)
    
    #fit model on short data
    model.fit(data_short)
    
    #output file name
    output_file_name = "/home/datascience/synthetic_from_db_" + table_name
    
    #delete file if exists already
    if os.path.exists(output_file_name):
        os.remove(output_file_name)
    else:
        print("The file does not exist yet, but that's fine")
    
    #create new synthetic rows and store as .csv file
    new_data = model.sample(num_rows = number_new_rows, output_file_path = output_file_name) 
    
    #load new data as pd dataframe
    new_csv_local_path = os.path.join("/home/datascience/", output_file_name)
    snapshot_synthetic_from_db = pd.read_csv(os.path.join("/home/datascience/", output_file_name))
    
    #push the new synthetic data as new table in the database
    snapshot_synthetic_from_db.ads.to_sql("TITANIC_SYNTHETIC",connection_parameters=connection_parameters, if_exists="replace")
    
    #show snapshot of synthetic data
    print("--------------------------------")
    print("--------------------------------")
    print("--------------------------------")
    print("Snapshot of the synthetic data is: ")
    print("--------------------------------")   
    
    #query synthetic data from the newly created table. And show.
    sql_statement_syn = "SELECT * FROM TITANIC_SYNTHETIC"
    df_synthetic = pd.DataFrame.ads.read_sql(sql_statement_syn, connection_parameters=connection_parameters)
    
    print(df_synthetic.head(10).to_string())
    
    print("--------------------------------")
    print("--------------------------------")
    return print("Synthetic data has been generated and pushed to database")


## 2.2 Run script and see output

In [ ]:
# determine values to pass 
primary_key = "PASSENGERID"
table_name = "TITANIC_ORIGINAL"
number_new_rows = 200

#call script_2
data = {"prim_key":primary_key, "table_name":table_name, "number_new_rows":number_new_rows}
script_2(data)